# Backtest

Two input:

1. from csv

2. give tickers

In [ ]:
# Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm

from func_cointegration import calculate_zscore

In [ ]:
# Get updated close prices
df_markets = pd.read_csv('1_all_markets_close_prices.csv')
df_markets

## 1. from csv
### Engineer DataFrame

In [ ]:
# Retrieve Co-integrated Pairs

# df_coint = pd.read_csv('2_cointegrated_pairs.csv')
# df_coint

In [ ]:
# Work on first row in df_coint
# x = 0 

# # Get tickers from the first row
# base_market = df_coint.iloc[x]['base_market']
# quote_market = df_coint.iloc[x]['quote_market']


## 2. give tickers

In [ ]:
base_market = 'SUSHI-USD'
quote_market = 'ICP-USD'

## The following is same for both options

In [ ]:
# Create a new df with prices of the base and quote market
df = df_markets[[base_market, quote_market]]

# Add datetime as index
df.index = df_markets['datetime']
df

In [ ]:
# Retrieve both series
series_1 = df[base_market].values
series_2 = df[quote_market].values
# series_1 = df_markets[base_market].values
# series_2 = df_markets[quote_market].values
series_1


In [ ]:
# Calculate the hedge ratio and the spread
model = sm.OLS(series_1, series_2).fit()
hedge_ratio = model.params[0]
spread = series_1 - (hedge_ratio * series_2)
spread

In [ ]:
# Calculate the z-score
zscore = calculate_zscore(spread) # Proved to be working!! :)
zscore

In [ ]:
# Add the spread and z-score to the dataframe
df.loc[:, 'spread'] = spread
df.loc[:, 'zscore'] = zscore.values
df

In [ ]:
# Drop nan rows
df = df.dropna()
df

In [ ]:
# Add zscore_lag for shift(1) to avoid look ahead bias
df.loc[:, 'zscore_lag'] = df['zscore'].shift(1)
# Make zscore_lag[0] = zscore_lag[1]
df.loc[df.index[0], 'zscore_lag'] = df['zscore_lag'].iloc[1]
df
 

### Calculate PnL & ROI

In [ ]:


zscore_entry = 1.5
zscore_exit = 1.49
trade_fee = 0.05 / 100
fund_portion_on_base = 0.5 # between 0 and 1 

initial_capital = 1000 # fixed
bet_on_base = initial_capital * fund_portion_on_base
bet_on_quote = initial_capital * (1 - fund_portion_on_base)

base_long_entry_price = 0
base_short_entry_price = 0
quote_long_entry_price = 0
quote_short_entry_price = 0

base_long_position = 0
base_short_position = 0
quote_long_position = 0
quote_short_position = 0

base_long_pnl = 0 # long: exit price - entry price
base_short_pnl = 0 # short: entry price - exit price
quote_long_pnl = 0
quote_short_pnl = 0

base_long_pnl_pct = 0 
base_short_pnl_pct = 0 
quote_long_pnl_pct = 0
quote_short_pnl_pct = 0

unrealized_pnl = 0
realized_pnl = 0

base_long_pnl_series = [] 
base_short_pnl_series = [] 
quote_long_pnl_series = []
quote_short_pnl_series = []

base_long_pnl_pct_series = [] 
base_short_pnl_pct_series = [] 
quote_long_pnl_pct_series = []
quote_short_pnl_pct_series = []

base_long_position_series = []
base_short_position_series = []
quote_long_position_series = []
quote_short_position_series = []

unrealized_pnl_series = []
realized_pnl_series = []

roi_series = []

# loop through the rows in df
for index, row in df.iterrows():
    
    zscore_curr = row['zscore_lag']

    
    # If don't have positions -> consider zscore_entry
    if base_long_position == 0 and base_short_position == 0 and quote_long_position == 0 and quote_short_position == 0:
    
        # check abs(zscore_lag) >  zscore_entry
        if abs(zscore_curr) >= zscore_entry:
            
            # if zscore_curr < 0, buy base, sell quote
            if zscore_curr < 0:
                
                # Buy base
                base_long_position = (bet_on_base) / row[base_market]
                base_long_entry_price = row[base_market]
                fee = (base_long_position * base_long_entry_price) * trade_fee
                realized_pnl -= fee
                base_long_pnl = (row[base_market] - base_long_entry_price) * base_long_position
                base_long_pnl_pct = base_long_pnl / (base_long_entry_price * base_long_position)
                
                # Sell quote
                quote_short_position = (bet_on_quote) / row[quote_market]
                quote_short_entry_price = row[quote_market]
                fee = (quote_short_position * quote_short_entry_price) * trade_fee
                realized_pnl -= fee
                quote_short_pnl = (quote_short_entry_price - row[quote_market]) * quote_short_position
                quote_short_pnl_pct = quote_short_pnl / (quote_short_entry_price * quote_short_position)

                unrealized_pnl = base_long_pnl + quote_short_pnl
                

                
            # if zscore_curr >= 0, sell base, buy quote
            else:
                
                # Sell base
                base_short_position = (bet_on_base) / row[base_market]
                base_short_entry_price = row[base_market]
                fee = (base_short_position * base_short_entry_price) * trade_fee
                realized_pnl -= fee
                base_short_pnl = (base_short_entry_price - row[base_market]) * base_short_position
                base_short_pnl_pct = base_short_pnl / (base_short_entry_price * base_short_position)

                # Buy quote
                quote_long_position = (bet_on_quote) / row[quote_market]
                quote_long_entry_price = row[quote_market]
                fee = (quote_long_position * quote_long_entry_price) * trade_fee
                realized_pnl -= fee
                quote_long_pnl = (row[quote_market] - quote_long_entry_price) * quote_long_position
                quote_long_pnl_pct = quote_long_pnl / (quote_long_entry_price * quote_long_position)

                unrealized_pnl = base_short_pnl + quote_long_pnl   
    
    # If have positions -> consider zscore_exit
    else:
        
        # check abs(zscore_lag) >=  zscore_exit
        if abs(zscore_curr) >= zscore_exit:
            
            # if zscore_curr >= 0 
            if zscore_curr >= 0:
                
                # have positions need to exit
                if base_long_position > 0 and quote_short_position > 0:
                
                    # Exit base_long_position
                    fee = (base_long_position * row[base_market]) * trade_fee
                    realized_pnl -= fee
                    base_long_pnl = (row[base_market] - base_long_entry_price) * base_long_position
                    base_long_pnl_pct = base_long_pnl / (base_long_entry_price * base_long_position)
                    base_long_position = 0
                    base_long_entry_price = 0
                    
                    # Exit quote_short_position
                    fee = (quote_short_position * row[quote_market]) * trade_fee
                    realized_pnl -= fee
                    quote_short_pnl = (quote_short_entry_price - row[quote_market]) * quote_short_position
                    quote_short_pnl_pct = quote_short_pnl / (quote_short_entry_price * quote_short_position)
                    quote_short_position = 0
                    quote_short_entry_price = 0
                    
                    unrealized_pnl = 0
                    realized_pnl += (base_long_pnl + quote_short_pnl)
                    
                    base_long_pnl = 0
                    quote_short_pnl = 0
                
                # No positions need to exit
                else:
                    # update pnl (no fee)
                    if base_long_position > 0:
                        base_long_pnl = (row[base_market] - base_long_entry_price) * base_long_position
                        base_long_pnl_pct = base_long_pnl / (base_long_entry_price * base_long_position) 
                    
                    if base_short_position > 0:
                        base_short_pnl = (base_short_entry_price - row[base_market]) * base_short_position
                        base_short_pnl_pct = base_short_pnl / (base_short_entry_price * base_short_position)
                        
                    if quote_long_position > 0:
                        quote_long_pnl = (row[quote_market] - quote_long_entry_price) * quote_long_position
                        quote_long_pnl_pct = quote_long_pnl / (quote_long_entry_price * quote_long_position)
                        
                    if quote_short_position > 0:
                        quote_short_pnl = (quote_short_entry_price - row[quote_market]) * quote_short_position
                        quote_short_pnl_pct = quote_short_pnl / (quote_short_entry_price * quote_short_position)
                        
                    unrealized_pnl = base_long_pnl + quote_short_pnl + base_short_pnl + quote_long_pnl
                    

                
            # if zscore_curr < 0 
            else:
                # have positions need to exit
                if base_short_position > 0 and quote_long_position > 0:
                
                    fee = (base_short_position * row[base_market]) * trade_fee
                    realized_pnl -= fee
                    base_short_pnl = (base_short_entry_price - row[base_market]) * base_short_position
                    base_short_pnl_pct = base_short_pnl / (base_short_entry_price * base_short_position)
                    base_short_position = 0
                    base_short_entry_price = 0
                    
                    fee = (quote_long_position * row[quote_market]) * trade_fee
                    realized_pnl -= fee
                    quote_long_pnl = (row[quote_market] - quote_long_entry_price) * quote_long_position
                    quote_long_pnl_pct = quote_long_pnl / (quote_long_entry_price * quote_long_position)
                    quote_long_position = 0
                    quote_long_entry_price = 0
                    
                    unrealized_pnl = 0
                    realized_pnl += base_short_pnl + quote_long_pnl
                    
                    base_short_pnl = 0
                    quote_long_pnl = 0
                
                # No positions need to exit
                else:
                    # update pnl (no fee)
                    if base_long_position > 0:
                        base_long_pnl = (row[base_market] - base_long_entry_price) * base_long_position
                        base_long_pnl_pct = base_long_pnl / (base_long_entry_price * base_long_position) 
                    
                    if base_short_position > 0:
                        base_short_pnl = (base_short_entry_price - row[base_market]) * base_short_position
                        base_short_pnl_pct = base_short_pnl / (base_short_entry_price * base_short_position)
                        
                    if quote_long_position > 0:
                        quote_long_pnl = (row[quote_market] - quote_long_entry_price) * quote_long_position
                        quote_long_pnl_pct = quote_long_pnl / (quote_long_entry_price * quote_long_position)
                        
                    if quote_short_position > 0:
                        quote_short_pnl = (quote_short_entry_price - row[quote_market]) * quote_short_position
                        quote_short_pnl_pct = quote_short_pnl / (quote_short_entry_price * quote_short_position)
                        
                    unrealized_pnl = base_long_pnl + quote_short_pnl + base_short_pnl + quote_long_pnl
        
        # If have positions but zscore_exit not triggered -> update pnl (no fee)
        else:
            if base_long_position > 0:
                base_long_pnl = (row[base_market] - base_long_entry_price) * base_long_position
                base_long_pnl_pct = base_long_pnl / (base_long_entry_price * base_long_position) 
                
            if base_short_position > 0:
                base_short_pnl = (base_short_entry_price - row[base_market]) * base_short_position
                base_short_pnl_pct = base_short_pnl / (base_short_entry_price * base_short_position)
                
            if quote_long_position > 0:
                quote_long_pnl = (row[quote_market] - quote_long_entry_price) * quote_long_position
                quote_long_pnl_pct = quote_long_pnl / (quote_long_entry_price * quote_long_position)
                
            if quote_short_position > 0:
                quote_short_pnl = (quote_short_entry_price - row[quote_market]) * quote_short_position
                quote_short_pnl_pct = quote_short_pnl / (quote_short_entry_price * quote_short_position)
                
            unrealized_pnl = base_long_pnl + quote_short_pnl + base_short_pnl + quote_long_pnl
    
    roi = (realized_pnl + unrealized_pnl) / initial_capital
            
    base_long_pnl_series.append(base_long_pnl) 
    base_short_pnl_series.append(base_short_pnl) 
    quote_long_pnl_series.append(quote_long_pnl)
    quote_short_pnl_series.append(quote_short_pnl)

    base_long_pnl_pct_series.append(base_long_pnl_pct) 
    base_short_pnl_pct_series.append(base_short_pnl_pct) 
    quote_long_pnl_pct_series.append(quote_long_pnl_pct)
    quote_short_pnl_pct_series.append(quote_short_pnl_pct)
    
    base_long_position_series.append(base_long_position)
    base_short_position_series.append(base_short_position)
    quote_long_position_series.append(quote_long_position)
    quote_short_position_series.append(quote_short_position)

    unrealized_pnl_series.append(unrealized_pnl)
    realized_pnl_series.append(realized_pnl)
    
    roi_series.append(roi)
    


In [ ]:
# Add columns to df
df.loc[:, 'base_long_position'] = base_long_position_series
df.loc[:, 'quote_short_position'] = quote_short_position_series

df.loc[:, 'base_short_position'] = base_short_position_series
df.loc[:, 'quote_long_position'] = quote_long_position_series


df.loc[:, 'base_long_pnl'] = base_long_pnl_series
df.loc[:, 'quote_short_pnl'] = quote_short_pnl_series

df.loc[:, 'base_short_pnl'] = base_short_pnl_series
df.loc[:, 'quote_long_pnl'] = quote_long_pnl_series


df.loc[:, 'base_long_pnl_pct'] = base_long_pnl_pct_series
df.loc[:, 'quote_short_pnl_pct'] = quote_short_pnl_pct_series

df.loc[:, 'base_short_pnl_pct'] = base_short_pnl_pct_series
df.loc[:, 'quote_long_pnl_pct'] = quote_long_pnl_pct_series


df.loc[:, 'unrealized_pnl'] = unrealized_pnl_series
df.loc[:, 'realized_pnl'] = realized_pnl_series

df.loc[:, 'roi'] = roi_series

df.to_csv(f"backtesting_{base_market}_{quote_market}_{zscore_entry}_{zscore_exit}_{trade_fee * 100}_{fund_portion_on_base}.csv")

'''
zscore_entry = 1.5
zscore_exit = 1.49
trade_fee = 0.05 / 100
fund_portion_on_base = 0.5 # between 0 and 1 
'''

df